In [1]:
import os
import sqlite3

import google.generativeai as genai
from polars import read_parquet
from qdrant_client import QdrantClient

df = read_parquet("../data/interim/srag_2019_2024.parquet")
DATA_COLS = {col: str(df[col].dtype) for col in df.columns}
df.head()

/home/vitorduarte/Área de Trabalho/testes/Test-Indicium/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,…,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,DOSE_2REF,DOSE_ADIC,DOS_RE_BI,FAB_COV_1,FAB_COV_2,FAB_COVRF,FAB_COVRF2,FAB_ADIC,FAB_RE_BI,LOTE_1_COV,LOTE_2_COV,LOTE_REF,LOTE_REF2,LOTE_ADIC,LOT_RE_BI,FNT_IN_COV,TRAT_COV,TIPO_TRAT,DT_TRT_COV,OUT_TRAT,SURTO_SG,CO_DETEC,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
i64,f64,i64,f64,i64,cat,cat,f64,cat,f64,cat,f64,i64,i64,i64,i64,f64,cat,f64,cat,f64,cat,cat,f64,cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
315478195042,null,2,null,2,"""MG""","""BELO HORIZONTE""",1449.0,"""BELO HORIZONTE""",310620.0,"""M""",null,30,3,3030,6,1.0,null,2.0,"""BRASIL""",1.0,"""MG""","""BELO HORIZONTE""",1449.0,"""RIBEIRAO DAS NEVES""",315460.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
315478195276,null,1,null,1,"""SP""","""GVE I CAPITAL""",1331.0,"""SAO PAULO""",355030.0,"""F""",null,7,2,2007,6,1.0,null,5.0,"""BRASIL""",1.0,"""SP""","""GVE I CAPITAL""",1331.0,"""SAO PAULO""",355030.0,1.0,null,null,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
315478207219,null,1,null,1,"""PE""","""001""",1497.0,"""RECIFE""",261160.0,"""M""",null,1,3,3001,6,4.0,null,5.0,"""BRASIL""",1.0,"""PE""","""012""",5558.0,"""GOIANA""",260620.0,1.0,9.0,9.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
315478211086,null,2,null,2,"""SP""","""GVE XVII CAMPINAS""",1342.0,"""CAMPINAS""",350950.0,"""F""",null,5,2,2005,6,1.0,null,5.0,"""BRASIL""",1.0,"""SP""","""GVE XVII CAMPINAS""",1342.0,"""CAMPINAS""",350950.0,1.0,2.0,2.0,1.0,1.0,9.0,1.0,1.0,1.0,2.0,2.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
315478212765,null,2,null,2,"""PE""","""004""",1499.0,"""CARUARU""",260410.0,"""F""",null,3,2,2003,6,4.0,null,5.0,"""BRASIL""",1.0,"""PE""","""004""",1499.0,"""BELO JARDIM""",260170.0,1.0,9.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,9.0,9.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [2]:
class AIQueryAgent:
    """
    Um agente que recebe perguntas em linguagem natural,
    traduz para SQL, consulta um SQLite, sumariza os resultados
    e busca notícias relacionadas no Qdrant.
    """

    def __init__(
        self,
        sqlite_path: str,
        qdrant_url: str = "http://localhost:6333",
        qdrant_collection: str = "sars_cov_news",
        embedding_model: str = "models/embedding-001",
        sql_model: str = "gemini-2.5-flash",
        summarization_model: str = "gemini-2.5-flash",
        final_model: str = "gemini-2.5-flash",
    ) -> None:
        """
        :param sqlite_path: caminho para o arquivo .db do SQLite
        :param qdrant_url: URL do servidor Qdrant
        :param qdrant_collection: nome da coleção onde estão as notícias
        :param embedding_model: modelo de embedding do Gemini
        :param sql_model: modelo de LLM para tradução NL→SQL
        :param summarization_model: modelo de LLM para resumo de resultados
        :param final_model: modelo de LLM para montar a resposta final
        """
        self.sqlite_path = sqlite_path
        self.qdrant = QdrantClient(url=qdrant_url)
        self.qdrant_collection = qdrant_collection
        self.embedding_model = embedding_model
        self.sql_model = sql_model
        self.summarization_model = summarization_model
        self.final_model = final_model

        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

    def _execute_sql(self, sql: str) -> list[tuple]:
        """Executa a query SQL no banco SQLite e retorna todas as linhas."""
        with sqlite3.connect(self.sqlite_path) as conn:
            cur = conn.cursor()
            cur.execute(sql)
            return cur.fetchall()

    def _llm_chat(self, model: str, messages: list[dict]) -> str:
        """Chama o endpoint de chat do Gemini."""
        return genai.GenerativeModel(model).generate_content(
            "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        ).text.strip()

    def _natural_to_sql(self, question: str) -> str:
        """Traduz a pergunta em linguagem natural para uma consulta SQL válida."""
        return self._llm_chat(
            self.sql_model,
            [
                {"role": "system", "content": "You are a SQL query generator for SQLite."},
                {"role": "user", "content": f"Translate this question into SQL:\n{question}"},
            ],
        )

    def _summarize(self, question: str, results: list[tuple]) -> str:
        """Gera um pequeno resumo explicativo dos resultados obtidos."""
        return self._llm_chat(
            self.summarization_model,
            [
                {"role": "system", "content": "You summarize table results."},
                {
                    "role": "user",
                    "content": (
                        "Question:\n"
                        + question
                        + "\nResults (list of tuples):\n"
                        + repr(results)
                        + "\nWrite a short paragraph explaining what those data shows."
                    ),
                },
            ],
        )

    def _get_embedding(self, text: str) -> list[float]:
        """Gera embedding para um texto usando a API do Gemini."""
        return genai.embed_content(
            model=self.embedding_model, content=text
        )["embedding"]

    def _search_news(self, embedding: list[float], limit: int = 5) -> list[dict]:
        """
        Busca notícias relacionadas no Qdrant, usando cosine similarity.
        Retorna lista de hits (id, payload).
        """
        return [
            {"id": hit.id, "score": hit.score, "payload": hit.payload}
            for hit in self.qdrant.search(
                collection_name=self.qdrant_collection,
                query_vector=embedding,
                limit=limit,
                with_payload=True,
            )
        ]

    def ask(self, question: str) -> str:
        """Ciclo principal: recebe pergunta, executa SQL, resume, busca notícias e devolve resposta final."""
        sql = self._natural_to_sql(question)
        summary = self._summarize(question, self._execute_sql(sql))
        return self._llm_chat(
            self.final_model,
            [
                {"role": "system", "content": "You are an agent that answers questions based on tabular data and related news."},
                {
                    "role": "user",
                    "content": f"Question: {question}\n\n"
                    f"SQL generated:\n{sql}\n\n"
                    f"Data Summary:\n{summary}\n\n"
                    f"Related news (id and title):\n"
                    + "\n".join(
                        f"- {hit['id']}: {hit['payload'].get('title', '(sem título)')}"
                        for hit in self._search_news(self._get_embedding(summary))
                    ),
                },
            ],
        )

In [3]:
agent = AIQueryAgent(sqlite_path="../data/interim/srag_2019_2024.db")
print(agent.ask("what was the month with more cases of sars in 2019"))

E0000 00:00:1759112789.205555  181764 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 30.278007195s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 30
}
]